In [1]:
"""
🎯 투표 예측 모델 v5 (FE 교체 버전) - MLP + FT-Transformer 앙상블
===================================================
- 기존 v5 학습/모델 구조 유지
- 피처 엔지니어링만 (clean_data + build_features(train_df, test_df))로 교체
- Fold별로 train 기준 인코딩 유지 (train-val / train-test 각각 생성)
- Cat embedding은 LabelEncoded(age_group, gender, race, religion) 사용
"""

import os
import random
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

# ============================================================
# CONFIG
# ============================================================
SEED = 42
N_FOLDS = 5
BATCH_SIZE = 512
EPOCHS = 50
PATIENCE = 8
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 너가 항상 쓰는 기본 경로로 맞춤
TRAIN_PATH = "../../data/raw/train.csv"
TEST_PATH  = "../../data/raw/test_x.csv"

print(f"🖥️ Device: {DEVICE}")

def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

# ============================================================
# 데이터 로딩
# ============================================================
print("\n📂 데이터 로딩...")
train_raw = pd.read_csv(TRAIN_PATH)
test_raw  = pd.read_csv(TEST_PATH)

train_raw["voted_bin"] = (train_raw["voted"] == 2).astype(int)
print(f"Train: {train_raw.shape}, Test: {test_raw.shape}")

# ============================================================
# 이상치/무응답 처리 (사용자 제공 버전 그대로)
# ============================================================
def clean_data(df):
    df = df.copy()

    # 무응답 (0 → NaN -> Mean)
    for col in ["education", "engnat", "hand", "married", "urban"]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
            df[col] = df[col].fillna(df[col].mean())

    # familysize
    if "familysize" in df.columns:
        df.loc[df["familysize"] == 0, "familysize"] = np.nan
        df.loc[df["familysize"] > 15, "familysize"] = np.nan
        df["familysize"] = df["familysize"].fillna(df["familysize"].mean())

    # TP 0 → NaN -> Mean
    for col in [f"tp{i:02d}" for i in range(1, 11)]:
        if col in df.columns:
            df.loc[df[col] == 0, col] = np.nan
            df[col] = df[col].fillna(df[col].mean())

    # Q_E 클리핑
    for col in [f"Q{c}E" for c in "abcdefghijklmnopqrst"]:
        if col in df.columns:
            df[col] = df[col].clip(lower=100, upper=60000)

    return df

# ============================================================
# 피처 엔지니어링 (사용자 제공 버전 + fold별 train 기준 인코딩)
# ============================================================
def build_features(train_df, other_df):
    """
    - train_df 기준으로 LabelEncoding fit
    - other_df는 같은 encoder로 transform
    - 두 DF 모두 동일한 파생변수 생성/원본 drop
    """
    train = train_df.copy()
    other = other_df.copy()
    dataset = [train, other]

    # Q_A
    qa_cols = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]

    for data in dataset:
        # (컬럼이 없을 수도 있으니 안전 처리)
        def safe_get(col):
            return data[col] if col in data.columns else 0

        data["T"] = safe_get("QcA") - safe_get("QfA") + safe_get("QoA") - safe_get("QrA") + safe_get("QsA")
        data["V"] = safe_get("QbA") - safe_get("QeA") + safe_get("QhA") + safe_get("QjA") + safe_get("QmA") - safe_get("QqA")
        data["M"] = -safe_get("QkA")

    # 역채점 질문들
    flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
    for data in dataset:
        for flip in flipping_columns:
            if flip in data.columns:
                data[flip] = 6 - data[flip]

    # Secret 질문들
    flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
    for data in dataset:
        for flip in flipping_secret_columns:
            if flip in data.columns:
                data[flip] = 6 - data[flip]

    for data in dataset:
        # qa_cols가 다 있지 않으면 있는 것만 평균
        existing_qa = [c for c in qa_cols if c in data.columns]
        if len(existing_qa) > 0:
            data["Mach_score"] = data[existing_qa].mean(axis=1)
        else:
            data["Mach_score"] = 0.0

    # Q_E -> delay
    qe_cols = [f"Q{c}E" for c in "abcdefghijklmnopqrst"]
    for data in dataset:
        existing_qe = [c for c in qe_cols if c in data.columns]
        if len(existing_qe) > 0:
            data["delay"] = data[existing_qe].sum(axis=1)
            data["delay"] = data["delay"] ** (1 / 10)
        else:
            data["delay"] = 0.0

    # 원본 Q_A, Q_E 제거
    for data in dataset:
        drop_qa = [c for c in qa_cols if c in data.columns]
        drop_qe = [c for c in qe_cols if c in data.columns]
        if len(drop_qa) > 0:
            data.drop(drop_qa, axis=1, inplace=True)
        if len(drop_qe) > 0:
            data.drop(drop_qe, axis=1, inplace=True)

    # TP Big5
    for data in dataset:
        # 안전 계산 (없는 col은 0 처리)
        def col_or0(c): return data[c] if c in data.columns else 0
        data["extraversion"] = col_or0("tp01") - col_or0("tp06")
        data["agreeableness"] = col_or0("tp07") - col_or0("tp02")
        data["conscientiousness"] = col_or0("tp03") - col_or0("tp08")
        data["neuroticism"] = col_or0("tp04") - col_or0("tp09")
        data["openness"] = col_or0("tp05") - col_or0("tp10")

    # 10대 여부
    for data in dataset:
        data["teenager_ox"] = (data["age_group"].astype(str) == "10s").astype(int)

    # Label Encoding: train 기준 fit -> other transform
    needenco = ["age_group", "gender", "race", "religion"]
    for col in needenco:
        le = LabelEncoder()
        train_col = train[col].astype(str).fillna("NaN") if col in train.columns else pd.Series(["NaN"] * len(train))
        other_col = other[col].astype(str).fillna("NaN") if col in other.columns else pd.Series(["NaN"] * len(other))

        # other에 train에 없는 값이 있을 수 있으니 union으로 fit
        all_vals = pd.concat([train_col, other_col], axis=0).astype(str).values
        le.fit(all_vals)

        train[col] = le.transform(train_col)
        other[col] = le.transform(other_col)

    # 성별 상호작용
    for data in dataset:
        data["Es_gender"] = data["neuroticism"] * data["gender"]
        data["Con_gender"] = data["conscientiousness"] * data["gender"]
        data["Op_gender"] = data["openness"] * data["gender"]

    # drop list
    drop_list = ["index", "hand"]
    for data in dataset:
        for c in drop_list:
            if c in data.columns:
                data.drop(c, axis=1, inplace=True)

    # WR 일부만 유지
    wr_list = [f"wr_{i:02d}" for i in range(1, 14)]
    keep_wr = ["wr_01", "wr_03", "wr_06", "wr_09", "wr_11"]
    wr_no_need = [c for c in wr_list if c not in keep_wr]
    for data in dataset:
        for c in wr_no_need:
            if c in data.columns:
                data.drop(c, axis=1, inplace=True)

    return train, other

# ============================================================
# Dataset
# ============================================================
class TabDataset(Dataset):
    def __init__(self, X_num, X_cat, y=None):
        self.X_num = torch.tensor(X_num, dtype=torch.float32)
        self.X_cat = torch.tensor(X_cat, dtype=torch.long)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X_num)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X_num[idx], self.X_cat[idx]
        return self.X_num[idx], self.X_cat[idx], self.y[idx]

# ============================================================
# Model 1: MLP
# ============================================================
class MLP(nn.Module):
    def __init__(self, num_features, cat_dims, embed_dim=8, hidden_dims=[256, 128, 64], dropout=0.3):
        super().__init__()
        self.cat_dims = cat_dims
        self.embeddings = nn.ModuleList([nn.Embedding(dim + 1, embed_dim) for dim in cat_dims])

        input_dim = num_features + len(cat_dims) * embed_dim
        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
            ])
            prev_dim = hidden_dim

        self.mlp = nn.Sequential(*layers)
        self.output = nn.Linear(hidden_dims[-1], 1)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x_num, x_cat):
        if x_cat.shape[1] == 0:
            x = x_num
        else:
            cat_embeds = torch.cat([emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)], dim=1)
            x = torch.cat([x_num, cat_embeds], dim=1)
        x = self.mlp(x)
        return self.output(x)

# ============================================================
# Model 2: FT-Transformer (경량화)
# ============================================================
class NumericalEmbedding(nn.Module):
    def __init__(self, num_features, d_token):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(num_features, d_token) * 0.02)
        self.bias = nn.Parameter(torch.zeros(num_features, d_token))

    def forward(self, x):
        return x.unsqueeze(-1) * self.weight + self.bias

class FTTransformer(nn.Module):
    def __init__(self, num_features, cat_dims, d_token=48, n_layers=2, n_heads=4, dropout=0.2):
        super().__init__()
        self.num_embed = NumericalEmbedding(num_features, d_token)
        self.cat_embeds = nn.ModuleList([nn.Embedding(dim + 1, d_token) for dim in cat_dims])
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_token) * 0.02)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_token, nhead=n_heads, dim_feedforward=d_token * 2,
            dropout=dropout, activation="gelu", batch_first=True, norm_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(d_token),
            nn.Linear(d_token, d_token // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_token // 2, 1)
        )

    def forward(self, x_num, x_cat):
        num_tokens = self.num_embed(x_num)  # (B, num_f, d)

        if x_cat.shape[1] == 0:
            tokens = num_tokens
        else:
            cat_tokens = torch.stack([emb(x_cat[:, i]) for i, emb in enumerate(self.cat_embeds)], dim=1)  # (B, cat_f, d)
            tokens = torch.cat([num_tokens, cat_tokens], dim=1)

        cls = self.cls_token.expand(tokens.size(0), -1, -1)
        tokens = torch.cat([cls, tokens], dim=1)

        x = self.transformer(tokens)
        return self.head(x[:, 0])

# ============================================================
# 학습 함수
# ============================================================
def train_model(model, train_loader, val_loader, val_y, device, epochs=EPOCHS, patience=PATIENCE, lr=1e-3):
    model.to(device)

    pos_ratio = float(np.mean(val_y))
    pos_weight = torch.tensor([(1 - pos_ratio) / (pos_ratio + 1e-6)], device=device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=3)

    best_auc = 0.0
    best_state = None
    no_improve = 0

    for epoch in range(epochs):
        model.train()
        for X_num, X_cat, y in train_loader:
            X_num, X_cat, y = X_num.to(device), X_cat.to(device), y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(X_num, X_cat), y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        model.eval()
        val_preds = []
        with torch.no_grad():
            for X_num, X_cat, _ in val_loader:
                X_num, X_cat = X_num.to(device), X_cat.to(device)
                val_preds.append(torch.sigmoid(model(X_num, X_cat)).cpu().numpy())

        val_preds = np.concatenate(val_preds).ravel()
        val_auc = roc_auc_score(val_y, val_preds)
        scheduler.step(val_auc)

        if val_auc > best_auc + 1e-5:
            best_auc = val_auc
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= patience:
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, best_auc

def predict(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in loader:
            X_num, X_cat = batch[0].to(device), batch[1].to(device)
            preds.append(torch.sigmoid(model(X_num, X_cat)).cpu().numpy())
    return np.concatenate(preds).ravel()

# ============================================================
# 메인
# ============================================================
def main():
    print("\n" + "="*60)
    print("🚀 MLP + FT-Transformer 앙상블 학습 (FE 교체 버전)")
    print("="*60)

    set_seed(SEED)

    # clean은 전체에 1번 적용해도 되지만,
    # fold별로 train/val/test 분리 후에도 동일하게 적용되도록 fold 내부에서 한 번 더 적용함
    train_base = train_raw.copy()
    test_base  = test_raw.copy()

    oof_mlp = np.zeros(len(train_base))
    oof_ft  = np.zeros(len(train_base))
    test_mlp = np.zeros(len(test_base))
    test_ft  = np.zeros(len(test_base))

    fold_aucs_mlp, fold_aucs_ft = [], []

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

    for fold, (tr_idx, va_idx) in enumerate(skf.split(train_base, train_base["voted_bin"])):
        print(f"\n{'='*50}")
        print(f"📂 Fold {fold+1}/{N_FOLDS}")
        print(f"{'='*50}")

        train_fold = train_base.iloc[tr_idx].copy().reset_index(drop=True)
        val_fold   = train_base.iloc[va_idx].copy().reset_index(drop=True)
        test_fold  = test_base.copy().reset_index(drop=True)

        # clean
        train_fold = clean_data(train_fold)
        val_fold   = clean_data(val_fold)
        test_fold  = clean_data(test_fold)

        # FE (train 기준)
        train_fe, val_fe = build_features(train_fold, val_fold)
        _, test_fe       = build_features(train_fold, test_fold)

        # voted_bin 보존
        train_fe["voted_bin"] = train_fold["voted_bin"].values
        val_fe["voted_bin"]   = val_fold["voted_bin"].values

        # cat / num 정의
        cat_features = ["age_group", "gender", "race", "religion"]
        cat_features = [c for c in cat_features if c in train_fe.columns]  # 혹시 없으면 제외
        num_features = [c for c in train_fe.columns if c not in (["voted", "voted_bin"] + cat_features)]

        # numeric matrix
        X_train = train_fe[num_features].copy()
        X_val   = val_fe[num_features].copy()
        X_test  = test_fe[num_features].copy()

        # NaN -> median
        for col in num_features:
            med = X_train[col].median()
            if pd.isna(med):
                med = 0.0
            X_train[col] = X_train[col].fillna(med)
            X_val[col]   = X_val[col].fillna(med)
            X_test[col]  = X_test[col].fillna(med)

        scaler = StandardScaler()
        X_train_s = scaler.fit_transform(X_train.values)
        X_val_s   = scaler.transform(X_val.values)
        X_test_s  = scaler.transform(X_test.values)

        # cat matrix
        if len(cat_features) == 0:
            X_cat_train = np.zeros((len(train_fe), 0), dtype=np.int64)
            X_cat_val   = np.zeros((len(val_fe), 0), dtype=np.int64)
            X_cat_test  = np.zeros((len(test_fe), 0), dtype=np.int64)
            cat_dims = []
        else:
            X_cat_train = train_fe[cat_features].fillna(0).astype(int).values
            X_cat_val   = val_fe[cat_features].fillna(0).astype(int).values
            X_cat_test  = test_fe[cat_features].fillna(0).astype(int).values

            # dim = max+1 (train/val/test 통합 기준)
            cat_dims = []
            for j in range(X_cat_train.shape[1]):
                m = int(max(X_cat_train[:, j].max(), X_cat_val[:, j].max(), X_cat_test[:, j].max()))
                cat_dims.append(m + 1)

        y_train = train_fe["voted_bin"].values.astype(np.float32)
        y_val   = val_fe["voted_bin"].values.astype(np.float32)

        # DataLoader
        train_ds = TabDataset(X_train_s, X_cat_train, y_train)
        val_ds   = TabDataset(X_val_s, X_cat_val, y_val)
        test_ds  = TabDataset(X_test_s, X_cat_test)

        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
        val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
        test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

        # =====================
        # Model 1: MLP
        # =====================
        print("\n  🔷 MLP 학습...")
        mlp = MLP(
            num_features=X_train_s.shape[1],
            cat_dims=cat_dims,
            embed_dim=8,
            hidden_dims=[256, 128, 64],
            dropout=0.3
        )
        mlp, auc_mlp = train_model(mlp, train_loader, val_loader, y_val, DEVICE, lr=1e-3)
        fold_aucs_mlp.append(auc_mlp)
        print(f"    MLP AUC: {auc_mlp:.5f}")

        oof_mlp[va_idx] = predict(mlp, val_loader, DEVICE)
        test_mlp += predict(mlp, test_loader, DEVICE) / N_FOLDS

        # =====================
        # Model 2: FT-Transformer
        # =====================
        print("\n  🔶 FT-Transformer 학습...")
        ft = FTTransformer(
            num_features=X_train_s.shape[1],
            cat_dims=cat_dims,
            d_token=48,
            n_layers=2,
            n_heads=4,
            dropout=0.2
        )
        ft, auc_ft = train_model(ft, train_loader, val_loader, y_val, DEVICE, lr=5e-4)
        fold_aucs_ft.append(auc_ft)
        print(f"    FT AUC: {auc_ft:.5f}")

        oof_ft[va_idx] = predict(ft, val_loader, DEVICE)
        test_ft += predict(ft, test_loader, DEVICE) / N_FOLDS

    # =====================
    # 앙상블
    # =====================
    print("\n" + "="*60)
    print("🎯 앙상블 결과")
    print("="*60)

    oof_auc_mlp = roc_auc_score(train_base["voted_bin"], oof_mlp)
    oof_auc_ft  = roc_auc_score(train_base["voted_bin"], oof_ft)

    print(f"\n📊 MLP:")
    print(f"   OOF AUC: {oof_auc_mlp:.5f}")
    print(f"   Fold AUCs: {[f'{x:.5f}' for x in fold_aucs_mlp]}")

    print(f"\n📊 FT-Transformer:")
    print(f"   OOF AUC: {oof_auc_ft:.5f}")
    print(f"   Fold AUCs: {[f'{x:.5f}' for x in fold_aucs_ft]}")

    oof_ensemble = (oof_mlp + oof_ft) / 2
    test_ensemble = (test_mlp + test_ft) / 2
    oof_auc_ensemble = roc_auc_score(train_base["voted_bin"], oof_ensemble)

    print(f"\n🏆 앙상블 (MLP + FT 평균):")
    print(f"   OOF AUC: {oof_auc_ensemble:.5f}")

    best_w = 0.5
    best_auc = oof_auc_ensemble
    for w in [0.3, 0.4, 0.5, 0.6, 0.7]:
        oof_w = w * oof_mlp + (1 - w) * oof_ft
        auc_w = roc_auc_score(train_base["voted_bin"], oof_w)
        if auc_w > best_auc:
            best_auc = auc_w
            best_w = w

    print(f"\n🏆 최적 가중 앙상블 (MLP:{best_w:.1f} + FT:{1-best_w:.1f}):")
    print(f"   OOF AUC: {best_auc:.5f}")

    final_test = best_w * test_mlp + (1 - best_w) * test_ft

    sub = pd.DataFrame({
        "index": test_raw["index"] if "index" in test_raw.columns else range(len(test_raw)),
        "voted": final_test
    })
    sub.to_csv("submission_21_v5_fe_swap_ensemble.csv", index=False)

    print(f"\n💾 저장: submission_21_v5_fe_swap_ensemble.csv")
    print(f"   예측 범위: [{final_test.min():.4f}, {final_test.max():.4f}]")

    # 개별 저장
    pd.DataFrame({"index": sub["index"], "voted": test_mlp}).to_csv("submission_21_v5_fe_swap_mlp.csv", index=False)
    pd.DataFrame({"index": sub["index"], "voted": test_ft}).to_csv("submission_21_v5_fe_swap_ft.csv", index=False)
    print("   (개별 저장: submission_21_v5_fe_swap_mlp.csv, submission_21_v5_fe_swap_ft.csv)")

    return best_auc

if __name__ == "__main__":
    main()


🖥️ Device: cpu

📂 데이터 로딩...
Train: (45532, 79), Test: (11383, 77)

🚀 MLP + FT-Transformer 앙상블 학습 (FE 교체 버전)

📂 Fold 1/5

  🔷 MLP 학습...
    MLP AUC: 0.77493

  🔶 FT-Transformer 학습...
    FT AUC: 0.77680

📂 Fold 2/5

  🔷 MLP 학습...
    MLP AUC: 0.76664

  🔶 FT-Transformer 학습...
    FT AUC: 0.76755

📂 Fold 3/5

  🔷 MLP 학습...
    MLP AUC: 0.76235

  🔶 FT-Transformer 학습...
    FT AUC: 0.75922

📂 Fold 4/5

  🔷 MLP 학습...
    MLP AUC: 0.76007

  🔶 FT-Transformer 학습...
    FT AUC: 0.75820

📂 Fold 5/5

  🔷 MLP 학습...
    MLP AUC: 0.76783

  🔶 FT-Transformer 학습...
    FT AUC: 0.76889

🎯 앙상블 결과

📊 MLP:
   OOF AUC: 0.76620
   Fold AUCs: ['0.77493', '0.76664', '0.76235', '0.76007', '0.76783']

📊 FT-Transformer:
   OOF AUC: 0.76559
   Fold AUCs: ['0.77680', '0.76755', '0.75922', '0.75820', '0.76889']

🏆 앙상블 (MLP + FT 평균):
   OOF AUC: 0.76917

🏆 최적 가중 앙상블 (MLP:0.5 + FT:0.5):
   OOF AUC: 0.76917

💾 저장: submission_21_v5_fe_swap_ensemble.csv
   예측 범위: [0.1206, 0.9940]
   (개별 저장: submission_21_v5_fe_swap_ml